In [1]:
import pickle
import pandas as pd
import numpy as np


from sklearn.utils import resample


df = pickle.load( open( "files/df.pickle", "rb" ) )
df['Tweet'] = df['Tweet'].astype(str)
df['Words'] = df['Tweet'].str.count(' ') + 1
df = df[df['Words']>=4]

cv = pickle.load( open( "models/countVectorizer1.pickle", "rb" ) )
lda = pickle.load( open( "models/lda_model.pickle", "rb" ) )
print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 5933852
Russian 1770487


In [2]:
df_majority = df[df['Category']==0]
df_minority = df[df['Category']==1]
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=df_minority.shape[0],     # to match minority class
                                 random_state=123) # reproducible results

df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [3]:

ldaVector = lda.transform(cv.transform(df_downsampled['Processed']))

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(ldaVector, df_downsampled['Category'])

In [5]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
print(model.score(X_train, y_train), model.score(X_test, y_test))

0.671523460592756 0.6716137019849895


In [ ]:
pickle.dump( model, open( "files/model.pickle", "wb" ) )

In [ ]:
model.coef_.shape